In [ ]:
class MCTS:
    def __init__(self, root_state, max_iterations):
        self.root = Node(root_state)
        self.max_iterations = max_iterations

    def selection(self, node):
        pass

    def expansion(self, node):
        pass

    def simulation(self, node):
        pass

    def backpropagation(self, node, reward):
        pass

    def search(self):
        for _ in range(self.max_iterations):
            node = self.root

            # selection phase
            while not node.is_terminal() and len(node.children) == len(node.state.get_legal_actions()):
                node = self.selection(node)

            # expansion phase
            if not node.is_terminal():
                node = self.expansion(node)

            # simulation phase
            reward = self.simulation(node)

            # backpropagation phase
            self.backpropagation(node, reward)

        # select the best action to take from the root node
        best_action = self.select_best_action(self.root)

        return best_action

    def select_best_action(self, node):
        # return the action with the highest average reward from the root node
        pass